In [1]:
# Neural Network Template for ReverseLearning with Input Backprop
# By Abraham Oliver, 2016

In [2]:
import tensorflow as tf
import numpy as np

# Import python3's print as a function
from __future__ import print_function

In [3]:
"""
Import data and prepare for use

ARGUMENTS:
    size - int - number of data in set

RETURN:
    ([input1, input2, input3, ...], [output_label1, output_label2, output_label3, ...])
    where input(n) is in form [value1, value2, value3, ...]
    and output_label(n) is in form [output_node1, output_node2, output_node3, ...]
    
NOTES:
    - DO NOT CHANGE NAME
    - Random import can be removed if not used
"""
from random import randint, random
def newSet(size):
    """EXAMPLE"""
    data = []
    labels = []
    for s in range(size):
        newInputs = [random() * randint(-10, 10) for i in range(3)]
        data.append(newInputs)
        labels.append([sum(newInputs)])
    return (data, labels)

In [4]:
# Start an interactive session
sess = tf.InteractiveSession()

In [5]:
# PARAMETERS (DESIGN AT WILL)

# Number of neurons in each layer where LAYERS[0] is the input and LAYERS[-1] is the output
LAYERS = [3, 1]

# HYPERPARAMETERS
LEARN_RATE = .01
EPOCHS = 4000
BATCH_SIZE = 200

# To change LOSS, go to cell 6
# To change OPTIMIZER, go to cell 6

# INPUT BACKPROP
IB_EPOCHS = 400
IB_LEARN_RATE = .01
TARGET = [10.0]

# To change LOSS for input backprop, go to cell 12
# To change OPTIMIZER for input backprop, go to cell 12

# Optional Input for prediction checking
EXAMPLE = []

# SETTINGS
DEBUG = False
DEBUG_INTERVAL = 2000

In [6]:
# Define Model Parameters (CUSTOMIZATION NOT NEEDED)

# Input
x = tf.placeholder(tf.float32, [None, LAYERS[0]], name="x")
# Weights
w = [tf.Variable(tf.zeros([LAYERS[n], LAYERS[n + 1]]), name="w{0}".format(n)) for n in range(len(LAYERS) - 1)]
# Biases
b = [tf.Variable(tf.ones([LAYERS[n + 1]]), name="b{0}".format(n)) for n in range(len(LAYERS) - 1)]
# Output
def calc(inp, n = 0):
    if n == len(LAYERS) - 2:
        return tf.matmul(inp, w[n]) + b[n]
    return calc(tf.matmul(inp, w[n]) + b[n], n + 1)
y = calc(x)
# Label
y_ = tf.placeholder(tf.float32, [None, LAYERS[-1]], name="y_")

# Loss function
loss = tf.reduce_mean(tf.pow(y_ - y, 2))
# Training step
train_step = tf.train.ProximalGradientDescentOptimizer(LEARN_RATE).minimize(loss)

In [7]:
# Train model (CUSTOMIZATION NOT NEEDED)

# Initialize variables
sess.run(tf.initialize_all_variables())

# Status bar
STATUS_INTERVAL = EPOCHS / 10

# Train normal model
print("TRAINING", end="")
for i in range(EPOCHS):
    # Get data
    batch_inps, batch_outs = newSet(BATCH_SIZE)
    
    # Debug printing
    if i % DEBUG_INTERVAL == 0 and DEBUG:
        print("Weights ::")
        for i in w:
            print(i.eval())
        print("Biases ::")
        for i in b:
            print(i.eval())
        print("Loss :: {0}\n\n".format(loss.eval(feed_dict={x: batch_inps, y_: batch_outs})))
        
    # Run train step
    sess.run(train_step, feed_dict={x: batch_inps, y_: batch_outs})
    
    # Print status bar
    if i % STATUS_INTERVAL == 0 and not DEBUG: print(" * ", end="")
print("\nTRAINING COMPLETE")

TRAINING *  *  *  *  *  *  *  *  *  * 
TRAINING COMPLETE


In [8]:
# Use trained network
def predict(INPUT):
    """
    Get network prediction
    
    ARGUMENTS
        INPUT - input vector. FORM: [[x0, x1, x2, ..., x(n-1)]] for n inputs
        full - bool - Return full vector output if true and only argmax if false
        
    EDIT LINE 2 TO CUSTOMIZE PREDICTOR
    """
    return calc(INPUT).eval()

if EXAMPLE != []:
    print("EXAMPLE :: {0} => {1}".format(EXAMPLE, predict(EXAMPLE)))

In [9]:
# Save variables
saved_weights = [i.eval() for i in w]
saved_biases = [i.eval() for i in b]
sess.close()

In [10]:
# Start new session
sess = tf.InteractiveSession()

In [11]:
# Input
optimal = tf.Variable(tf.zeros([1, LAYERS[0]]))
# Input Weights
w = [tf.constant(i) for i in saved_weights]
# Input Biases
b = [tf.constant(i) for i in saved_biases]
# Output
out = calc(optimal)
# Label
lbl = tf.constant(TARGET)

# Training with quadratic cost and gradient descent with learning rate .01
loss = tf.pow(tf.reduce_mean(lbl - out), 2)
train_step = tf.train.ProximalGradientDescentOptimizer(LEARN_RATE).minimize(loss)

# Initialize variables
sess.run(tf.initialize_all_variables())

# Train to find three inputs
for i in range(IB_EPOCHS):
    sess.run(train_step)

print("OPTIMAL INPUT       :: {0}".format(optimal.eval()))
print("CALCULATED OUT      :: {0}".format(calc(optimal.eval()).eval()))
print("TARGET OUT          :: {0}".format(TARGET))
print("TARGET vs CALC LOSS :: {0}".format(loss.eval()))

OPTIMAL INPUT       :: [[ 3.33333135  3.33333135  3.33333135]]
CALCULATED OUT      :: [[ 9.99999428]]
TARGET OUT          :: [10.0]
TARGET vs CALC LOSS :: 3.27418092638e-11
